This notebook is intended to solve the "general" problem of steering in the linac by solving for correctors which give the optimal emittance and steering at some final checkpoint

Practically, the purpose here is to give a final refinement to a one-to-one steering solution produced by, e.g. "2025-06-12 Creating config files for misalignments and steering solutions.ipynb"

In [1]:
from UTILITY_quickstart import *

In [2]:
importedDefaultSettings = loadConfig("setLattice_configs/2024-10-14_twoBunch_baseline.yml")
#importedDefaultSettings = loadConfig("setLattice_configs/2024-10-22_oneBunch_baseline2.yml")

#New DL10 centering 
#importedDefaultSettings = importedDefaultSettings | {"centerDL10" : True}

In [3]:
csrTF = False
evalElement = "PENT"


inputBeamFilePathSuffix = importedDefaultSettings["inputBeamFilePathSuffix"]
bunchCount = importedDefaultSettings["bunchCount"]
tao = initializeTao(
    inputBeamFilePathSuffix = inputBeamFilePathSuffix,
    
    csrTF = csrTF,
    numMacroParticles=1e3,
    scratchPath = "/tmp",
    randomizeFileNames = True
)

Environment set to:  /Users/nmajik/Documents/SLAC/FACET2-Bmad-PyTao
Tracking to end
CSR off
Overwriting lattice with setLattice() defaults
No defaults file provided to setLattice(). Using setLattice_configs/defaults.yml
Number of macro particles = 1000.0
Beam created, written to /tmp/beams/activeBeamFile_1316998700231803726.h5, and reinit to tao


In [4]:
setLattice(tao, **importedDefaultSettings)

#with open('./other_configs/pinkCurveMisalignments.json', 'r') as f:
#with open('./other_configs/blueCurveMisalignments.json', 'r') as f:
with open('./other_configs/winter2025BBAMisalignments.json', 'r') as f:
    misalignmentConfig = json.load(f)

applyOtherConfig(tao, misalignmentConfig) 



# Import the one-to-one steering config as a starting point
#with open('./other_configs/2025-06-13_pinkCurveSteering_2024-10-14_twoBunch_baseline.json', 'r') as f:
#with open('./other_configs/2025-06-13_blueCurveSteering_2024-10-14_twoBunch_baseline.json', 'r') as f:
with open('./other_configs/2025-06-13_winter2025BBASteering_2024-10-14_twoBunch_baseline.json', 'r') as f:
    importedSteeringConfig = json.load(f)

#Optional: Use one-to-one steering for most correctors
applyOtherConfig(tao, importedSteeringConfig) 

In [5]:
totalNumElements = len(tao.lat_list("*", "ele.name"))
combinedElementData = [
    {
        "eleII" : eleII,
        "name" : tao.ele_head(eleII)["name"],
        "key" : tao.ele_head(eleII)["key"],
        "s" : tao.ele_head(eleII)["s"]
    }
    for eleII in range(totalNumElements) 
]

#Keep only elements after L0AFEND
combinedElementData = [ele for ele in combinedElementData if ele["s"] > 4.1274477]

allHKickerNames = [ele["name"] for ele in combinedElementData if ele["key"] == "HKicker"]
allVKickerNames = [ele["name"] for ele in combinedElementData if ele["key"] == "VKicker"]



######################################
#Optional: Limit by s

#optimizerKickerSMin = tao.ele_head("ENDDL10")["s"]
optimizerKickerSMin = tao.ele_head(importedSteeringConfig[-1][0])["s"] #Start from the last kicker in the imported steering configuration
#optimizerKickerSMin = tao.ele_head(importedSteeringConfig[0][0])["s"] #Start from the first kicker in the imported steering configuration

optimizerKickerSMax = tao.ele_head("BEGBC20")["s"]

print( optimizerKickerSMin, optimizerKickerSMax ) 

allHKickerNames = [ ele for ele in allHKickerNames if optimizerKickerSMin < tao.ele_head(ele)["s"] < optimizerKickerSMax]
allVKickerNames = [ ele for ele in allVKickerNames if optimizerKickerSMin < tao.ele_head(ele)["s"] < optimizerKickerSMax]

combinedKickerNameList = allHKickerNames + allVKickerNames

879.809569853447 929.537144536861


In [6]:
def objective(params, tao):

    try: 
        configArr = [ [combinedKickerNameList[i], "BL_KICK", params[i]] for i in range(len(params)) ]
    
        applyOtherConfig(tao, configArr)
        
        trackBeam(tao, trackEnd = "BEGBC20", verbose = False)
    
        P = getBeamAtElement(tao, "BEGBC20")
    
        x = np.median(  P.x ) 
        xp = np.median( P["xp"] ) 
    
        y = np.median(  P.y ) 
        yp = np.median( P["yp"] )

        emitX = smallestIntervalImpliedEmittance(P, plane = "x", percentage = 0.90)
        emitY = smallestIntervalImpliedEmittance(P, plane = "y", percentage = 0.90)

        # x = np.mean(  P.x ) 
        # xp = np.mean( P["xp"] ) 
    
        # y = np.mean(  P.y ) 
        # yp = np.mean( P["yp"] )

        # emitX = (P.twiss(plane = "x", fraction = 0.9))["norm_emit_x"]
        # emitY = (P.twiss(plane = "y", fraction = 0.9))["norm_emit_y"]
    



    
        error = np.sqrt( x ** 2 + xp ** 2 + y ** 2 + yp ** 2 + emitX ** 2 + emitY ** 2 )
    
        #Optional: Give small preference to smaller corrector settings
        error = error + 1e-6 * np.linalg.norm(params)
    
        #print(params, error)
        print(error)
    
        return error

    except:
        print("Failed!")

        return 1e20

In [7]:
importedSteeringDict = { row[0] : row[2] for row in importedSteeringConfig }

In [8]:
from scipy.optimize import minimize

# initialGuess = [0 for name in combinedKickerNameList]
initialGuess = [ importedSteeringDict[key] if key in importedSteeringDict else 0 for key in combinedKickerNameList]



bounds = [(-0.02, 0.02) for ele in combinedKickerNameList]


# Perform optimization using Nelder-Mead
result = minimize(
    objective, 
    initialGuess, 
    method='Nelder-Mead',
    #method = "COBYQA",
    #method = "L-BFGS-B", 
    bounds = bounds,
    args = (tao, ),
    options = { "maxiter" : 10000 }
)

print("Optimization Results:")
print(f"Optimal Parameters: {result.x}")
print(f"Objective Function Value at Optimal Parameters: {result.fun}")
print(f"Number of Iterations: {result.nit}")
print(f"Converged: {result.success}")


0.00030663243460024066
0.00041065849868864626
0.0003604938492841307
0.00036359881923154325
0.0003259014353103077
0.00039453236454466405
0.00044040474717162517
0.0003672080764564661
0.00035333004307784967
0.0002985700769808235
0.0002930960542513216
0.0002871889039623129
0.0002950782871013458
0.0002651682345931636
0.000255757378983252
0.0002685380502094146
0.00020009252201980043
0.00011972806775130973
0.00014412641327010069
0.0002041970210992367
0.00017512010272299178
0.00025012379062744886
7.294547338765015e-05
8.958478030727904e-05
0.0003565369151431033
0.00019135689623369815
8.173978154136443e-05
4.378898172561687e-05
0.00014785545116615084
0.0001846513927073691
0.0001459203863634036
6.11508116732839e-05
0.00018188242398938388
0.00010476025770364595
0.0001292777203064132
0.00016551430121915906
7.921748011680334e-05
0.00010424936508085705
0.0001680670628015774
6.185096582651994e-05
0.0001336000887123306
6.407103192919513e-05
0.00010526921238194878
5.992971298702636e-05
0.00013066521781

In [9]:
result.x

array([-0.00016326,  0.00031846, -0.00043968,  0.00031776, -0.00085919,
       -0.00032083,  0.00039595,  0.00057689])

In [11]:


configArr = importedSteeringConfig + [ [combinedKickerNameList[i], "BL_KICK", result.x[i]] for i in range(len(result.x)) ]

# #with open('./other_configs/2025-06-13_pinkCurveSteering_2024-10-14_twoBunch_baseline_extended.json', 'w') as f:
# with open('./other_configs/2025-06-13_blueCurveSteering_2024-10-14_twoBunch_baseline_extended.json', 'w') as f:
#with open('./other_configs/2025-06-13_winter2025BBASteering_2024-10-14_twoBunch_baseline_extended.json', 'w') as f:
#     json.dump(configArr, f)
